In [2]:
import warnings
warnings.filterwarnings('ignore')

import os
import re
import json
import pandas as pd
import ipywidgets as widgets
from urllib.parse import parse_qs
from ipywidgets import AppLayout, Button, Layout
from IPython.display import display, Javascript, clear_output, HTML
from ipywidgets import GridspecLayout

In [3]:
# Function to get query parameters
def get_query_parameters():
    # Extract query parameters from the URL
    query_string = os.environ.get('QUERY_STRING', '')  # Voila sets this when rendering
    query_params = parse_qs(query_string)
    return query_params

# Get parameters
params = get_query_parameters()
filename = params.get("filename", ["Unknown"])[0]

In [4]:
# Create the directory path using the filename
dir_path = f"../user_data/json/{filename}"

# Create the full path for the JSON file inside the directory
path = os.path.join(dir_path, f"{filename}_storage.json")
back_path = os.path.join(dir_path, f"{filename}_backtracking.json")
ques_nav_path = os.path.join(dir_path, f"{filename}_ques_nav_list.json")

# Ensure the directory exists
os.makedirs(dir_path, exist_ok=True)

# Check if the JSON file exists
if not os.path.exists(path):
    # Create the JSON file with an empty object if it doesn't exist
    with open(path, 'w') as json_file:
        json.dump({}, json_file)  # Write an empty JSON object

if not os.path.exists(back_path):
    with open(back_path, 'w') as json_file:
        json.dump([], json_file)

In [5]:
%run ../02_ui_components/progressbar.ipynb

In [6]:
%run ../02_ui_components/sidebar.ipynb

In [7]:
%run ../02_ui_components/dropdown.ipynb

In [8]:
%run ../02_ui_components/header_button.ipynb

In [9]:
%run ../02_ui_components/more_info.ipynb

In [10]:
def get_css_class(css_file, class_name):
    with open(css_file, 'r') as file:
        css = file.read()
        # Use regex to extract the class
        pattern = r'\.' + re.escape(class_name) + r'\s*\{([^}]*)\}'
        match = re.search(pattern, css, re.DOTALL)
        if match:
            css_content = match.group(1).strip()
            return f'\n<style>\n.{class_name} {{\n{css_content}\n}}\n</style>\n'
        else:
            return None

In [11]:
css_path = "../css/style.css"

In [12]:
# Custom CSS to modify the question button design
ques_widget_css = get_css_class(css_path, 'ques-widget')

In [13]:
# Custom CSS to modify the option button design
opt_button_css = get_css_class(css_path, 'opt-button')

In [14]:
# Custom CSS to modify the header design
header_css = get_css_class(css_path, 'custom-header')

In [15]:
# Custom CSS to modify the footer design
footer_css = get_css_class(css_path, 'custom-footer')

In [16]:
# Create an HTML widget for the background color
background_color_html = widgets.HTML(
    value = get_css_class(css_path, 'custom-vbox')
)

In [17]:
# Taking from separate header notebook
header = header_prog1

In [18]:
# Creating a container for header button
header_container = widgets.VBox([header, header_widget], 
                    layout=widgets.Layout(width='95%', padding_top= '10px', align_items='flex-start', justify_content='center'))

In [19]:
# Creating button
def create_expanded_button(description, button_style):
    return Button(description=description, button_style=button_style, layout=Layout(height='auto', width='auto', border='1px solid black')) #width='max-content'

In [20]:
# Declaring necessary variables and importing neccesary files for storing user answers and keep track of the visited pages
global stored_choices, path
dropdown_path = "../data_files/csv/links.csv"
question_path = "../data_files/csv/question_data.csv"
button_color_path = "../data_files/csv/button_color.csv"

In [21]:
# Importing the json file for storing user answers
with open(path, "r") as f:
    stored_choices = json.load(f)

In [22]:
# Importing the json file for keeping track of visited pages
with open(back_path, 'r') as f:
    back_list = json.load(f)

In [23]:
# Reading the CSV data file into a DataFrame
df = pd.read_csv('../data_files/csv/question_data.csv', low_memory=False)

In [24]:
dp_df = pd.read_csv(dropdown_path, low_memory=False)

In [25]:
btn_color = pd.read_csv(button_color_path, low_memory=False)

In [26]:
drop_link = dp_df.loc[dp_df['field_code'] == 'num_cams', 'info_url'].iloc[0]

In [27]:
notebook_name = "01_03_num_cams"
page_dropdown_text = df.loc[df['page_name'] == 'num_cams', 'page_dropdown_text'].iloc[0]
notebook_list_name = "obj_list"
footer_name = "footer_in_prog1"

In [28]:
footer = main(filename, notebook_name, notebook_list_name, footer_name, page_dropdown_text)

TypeError: main() missing 1 required positional argument: 'page_dropdown_text'

In [29]:
# Assigns dynamic link to the last sidebar button based on the notebook dropdown info link
concept_grid[5, 0:9].on_click(on_concept_option_clicked(drop_link))

In [26]:
# seperates the string based on ',' also removes the whitespace after comma & makes a list
def split_and_trim(s):
    return [item.strip() for item in s.split(',')]

In [27]:
# gets the question based on value of other column

ques1 = df.loc[df['question_code'] == 'num_cams_limited', 'question_text'].iloc[0]
opt1 = df.loc[df['question_code'] == 'num_cams_limited', 'q_option_label'].iloc[0]
opt_codes1 = df.loc[df['question_code'] == 'num_cams_limited', 'q_option_code'].iloc[0]

opt1 = split_and_trim(opt1)
opt_codes1 = split_and_trim(opt_codes1)

# ques2 = df.loc[df['question_code'] == 'num_cams_avail', 'question_text'].iloc[0]

In [28]:
# widget on right side for more information
html_widget = more_info_widget("../data_files/markdown/01_02_objective_note.md")

In [29]:
# Factor widget

question1 =  f"""
    <div class="ques-widget">
        <p>{ques1}</p>
    </div>
    """
question1 = widgets.HTML(value=question1, layout=Layout(max_width='700px', padding='5px'))


option1 = create_expanded_button(opt1[0], 'success')
option1.style.button_color = btn_color.loc[btn_color['Category'] == 'Regular Button', 'Color_Code'].iloc[0]
option1.style.text_color = 'black'
option1.add_class('opt-button')

option2 = create_expanded_button(opt1[1], 'success')
option2.style.button_color = btn_color.loc[btn_color['Category'] == 'Regular Button', 'Color_Code'].iloc[0]
option2.style.text_color = 'black'
option2.add_class('opt-button')


options_widget2 = widgets.IntText(
                            value=0,  # Default value
                            description="Enter value",
                            disabled=False,
                            layout={'width': 'max-content'}
                        )


# Create a HTML widget to include the custom CSS
css_widget = widgets.HTML(value=ques_widget_css)
css_widget2 = widgets.HTML(value=opt_button_css)

factor_grid = GridspecLayout(8, 18, grid_gap='5px', height='550px')
factor_grid[3:6, 1:6] = question1
factor_grid[1:6, 9:18] = html_widget
factor_grid[3, 7:8] = option1
factor_grid[4, 7:8] = option2
factor_grid[5, 7:8] = options_widget2

# factor_grid

# logical_integer_container = widgets.VBox(children=[factor_grid, css_widget, css_widget2], layout=widgets.Layout(
#     align_items='center'  # Center align vertically
# #     border='solid 2px black',
# #     width = "70%"
# #     justify_content='center'  # Center align horizontally
# #     height='100vh'  # Set height to 100% of viewport height for vertical centering
# ))
# # logical_integer_container

SyntaxError: invalid syntax (3266215174.py, line 3)

In [31]:
# # Factor widget

#------------------------ uncomment for seperate questions-------------

# question2 = create_expanded_button(ques2, 'success')
# question2.style.button_color = 'lightblue'
# question2.style.text_color = 'black'
# # Add custom class to the button
# question2.add_class('ques-button')

# options_widget2 = widgets.IntText(
#                             value=0,  # Default value
#                             description="Enter value",
#                             disabled=False,
#                             layout={'width': 'max-content'}
#                         )

# # Create a HTML widget to include the custom CSS
# css_widget = widgets.HTML(value=ques_button_css)

# integer_grid = GridspecLayout(8, 12, grid_gap='5px', height='250px')
# integer_grid[3:5, 2:6] = question2
# integer_grid[3:5, 7:9] = options_widget2

#------------------------ uncomment for seperate questions-------------







# integer_container = widgets.VBox(children=[integer_grid, css_widget], layout=widgets.Layout(
#     align_items='center'  # Center align vertically
# #     border='solid 2px black',
# #     width = "70%"
# #     justify_content='center'  # Center align horizontally
# #     height='100vh'  # Set height to 100% of viewport height for vertical centering
# ))
# # integer_container

In [32]:
logical_integer_container = widgets.VBox(children=[factor_grid, css_widget, css_widget2], layout=widgets.Layout(
    align_items='center'  # Center align vertically   ### integer_grid
#     border='solid 2px black',
#     width = "70%"
#     justify_content='center'  # Center align horizontally
#     height='100vh'  # Set height to 100% of viewport height for vertical centering
))
# logical_integer_container

In [33]:
button_list = [option1, option2]

In [34]:
selected = False

In [35]:
output = widgets.Output()
def option_1_button_clicked(option1):
    with output:
        clear_output()
        global selected_objective
        global selected
        
        selected = True
        
        for btn in button_list:
            if btn == option1:
                btn.icon='check'
            else:
                btn.icon=''
        selected_option = factor_grid[3, 7:8].description
        selected_option_index = opt1.index(selected_option)
        selected_option_code = opt_codes1[selected_option_index]
        
        stored_choices["num_cams_limited"] = selected_option_code
        
        with open(path, "w") as f:
            json.dump(stored_choices, f)
        
#         display(integer_container) # OnClick displays
        if "objective" in stored_choices.keys():
            selected_objective = stored_choices['objective']
        
factor_grid[3, 7:8].on_click(option_1_button_clicked)

In [36]:
output = widgets.Output()
def option_2_button_clicked(option2):
    with output:
        clear_output()
        global selected_objective
        global selected
        
        selected = True
        
        for btn in button_list:
            if btn == option2:
                btn.icon='check'
            else:
                btn.icon=''
        
        selected_option = factor_grid[4, 7:8].description
        selected_option_index = opt1.index(selected_option)
        selected_option_code = opt_codes1[selected_option_index]
        
        stored_choices["num_cams_limited"] = selected_option_code
        stored_choices["num_cams_avail"] = 999999
        
        with open(path, "w") as f:
            json.dump(stored_choices, f)
            
        if "objective" in stored_choices.keys():
            selected_objective = stored_choices['objective']
        
factor_grid[4, 7:8].on_click(option_2_button_clicked)

In [37]:
accordion.observe(lambda change: show_popup(accordion, drop_link), names='selected_index')

NameError: name 'button' is not defined

In [49]:
# Create a buttons for submitting the question
submit_button = widgets.Button(description="Submit & Next >>", layout=widgets.Layout(border='1px solid black'))

In [50]:
# Create a button for going back to previous question
back_button = widgets.Button(description="<< Back", layout=widgets.Layout(border='1px solid black'))

In [51]:
# Create the text widget with a warning message
warning_widget = widgets.HTML(
    value="""
    <div style="padding: 5px; background-color: #f8d7da; color: #721c24; border: 2px solid #f5c6cb; border-radius: 5px;">
        <strong>Warning!</strong> Select an option first.
    </div>
    """
)

In [52]:
error = widgets.HBox(
    [warning_widget],
    layout=widgets.Layout(
        width='auto',  
        height='auto',
        margin='auto',  
        align_items='flex-end',  
        justify_content='flex-end' 
    )
)

In [53]:
# Create the text widget with a warning message
warning_widget2 = widgets.HTML(
    value="""
    <div style="padding: 5px; background-color: #f8d7da; color: #721c24; border: 2px solid #f5c6cb; border-radius: 5px;">
        <strong>Warning!</strong> Inappropriate answer.
    </div>
    """
)

In [54]:
error2 = widgets.HBox(
    [warning_widget2],
    layout=widgets.Layout(
        width='auto',  
        height='auto',
        margin='auto',  
        align_items='flex-end',  
        justify_content='flex-end' 
    )
)

In [55]:
# For going back to last question

output = widgets.Output()
def on_back_button_clicked(event):
    with output:
        clear_output()
        file_name = back_list[-1]
        notebook_filename = f"{file_name}.ipynb?filename={filename}"
        js_code = f"window.open('{notebook_filename}', '_self');"
#         link = Markdown('''<a href="{}" target="_self" rel="noopener noreferrer">Previous</a>'''.format(notebook_filename))
        back_list.pop()
        with open (back_path, "w")as f:
            json.dump(back_list, f)
        display(Javascript(js_code))

back_button.on_click(on_back_button_clicked)

In [56]:
# output = widgets.Output()
# def on_button_clicked(event):
#     with output:
#         clear_output()
        
#         if not selected:
#             display(error)
#         else:
#             condition = df.loc[df['page_name'] == 'cam_strat_covar', 'json_logic'].iloc[0]
#             condition_json = json.loads(condition)
#             if selected_objective in condition_json['objective']:
#                 next_notebook = "cam_strat_covar"
#             else:
#                 next_notebook = "surv_dur_min_max"

#             notebook_filename = next_notebook + ".ipynb"

#             # JavaScript to open the URL
#             js_code = f"window.open('{notebook_filename}', '_self');"


#             back_list.append("num_cams")
#             with open (back_path, "w")as f:
#                 json.dump(back_list, f)

#             if options_widget2.value > 0:
#                 stored_choices["num_cams_avail"] = options_widget2.value
#                 options_widget2.value = 0
#                 with open(path, "w") as f:
#                     json.dump(stored_choices, f)

#             display(Javascript(js_code))

# submit_button.on_click(on_button_clicked)

In [57]:
output = widgets.Output()
def on_button_clicked(event):
    with output:
#         clear_output()
        global stored_choices # <----NEEDED-----
        
        if not selected:
            display(error)
        else:
            condition = df.loc[df['page_name'] == 'cam_strat_covar', 'json_logic'].iloc[0]
            condition_json = json.loads(condition)
            if set(selected_objective) & set(condition_json['objective']):
                next_notebook = "01_06_cam_strat_covar"
            else:
                next_notebook = "01_08_surv_dur_min_max"

            notebook_filename = f"{next_notebook}.ipynb?filename={filename}"

            # JavaScript to open the URL
            js_code = f"window.open('{notebook_filename}', '_self');"


            back_list.append("01_03_num_cams")
            with open (back_path, "w")as f:
                json.dump(back_list, f)

            if options_widget2.value > 0:
                stored_choices["num_cams_avail"] = options_widget2.value

                with open(path, "w") as f:
                    json.dump(stored_choices, f)
            #--------------------------------------------->
            with open(path, "r") as f:
                stored_choices = json.load(f)
                
            if options_widget2.value <= 0 and stored_choices["num_cams_limited"] == "TRUE":
                display(error2)
            else:
                options_widget2.value = 0
                display(Javascript(js_code))

submit_button.on_click(on_button_clicked)

In [60]:
# Creating containers for all the widgets along with their css properties

all_button = widgets.HBox(
    [back_button, submit_button],
    layout=widgets.Layout(
        width='100%',  
        height='auto',
#         border='solid 2px grey',
        padding='1px',
        margin='auto',  
        align_items='center',  
        justify_content='space-between' 
    )
)
buttons = widgets.VBox(
    [output, all_button],
    layout=widgets.Layout(
        width='100%',  
        height='auto',  
#         border='solid 2px grey',
        padding='1px',
        margin='auto',  
        align_items='center',  
        justify_content='center'  
    )
)



upper = widgets.VBox(
    [logical_integer_container, buttons],
    layout=widgets.Layout(
        width='95%',  
        min_height= '580px',
        height='auto', 
        border='solid 1px grey',
        padding='10px',
        margin='auto',  
        align_items='center',  
        justify_content='center'
    )
)
upper.add_class("custom-vbox")

popup = widgets.VBox(children=[accordion], layout=widgets.Layout(
        width='95.3%',
        padding ='5px 0px 0px 0px'))


right_col = widgets.VBox(children=[header_container, background_color_html, upper, footer, popup], layout=widgets.Layout(
        width='88%',  
        height='100%',  
        border_left='solid 1px grey',
        padding='5px',
        margin='auto',  
        align_items='center',  
        justify_content='center'
))

cols = widgets.HBox(
    [left_col, right_col],
    layout=widgets.Layout(
        width='95%',  
        height='auto',  
        border='solid 2px grey',
#         padding='5px',
        margin='auto',  
        align_items='flex-start', 
        justify_content='center'
    )
)

display(cols)